In [2]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import re

import pandas as pd
import numpy as np


# from google.colab import files


# 2k ratings data


In [17]:
# base URL of the site
url="https://nba2k19.2kratings.com"


# robots.txt

# def check_robots_txt(url):
#   url_robots = url + '/robots.txt'
#   page = requests.get(url_robots)
#   soup = BeautifulSoup(page.content, 'lxml')
#   print(soup.prettify())
  
  
# check_robots_txt(url) 

In [40]:
url="https://nba2k19.2kratings.com/team/atlanta-hawks"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
table = soup.find("tbody")

for player in table.find_all("tr"):

    print(player.find_all("td"))
    # name of the player
    name = player.find_all("td")[1].a.text
    pos = player.find_all("td")[3].text
#     team = player.find_all("td")[1].small.text
    # height
    height =  player.find_all("td")[4].text

    # rating
    rating = player.find_all("td")[2].span.text

    # brand
    brand = player.find_all("td")[5].text

    print(name, pos, height, brand, rating )
    ratings_19 = pd.DataFrame({'name': [name], 'rating': [rating], 'team': ["ATL"]})
ratings_19

[<td class="hidden-xs font-weight-normal">24</td>, <td class="roster-entry"><a href="https://nba2k19.2kratings.com/player/kent-bazemore"><img alt="Kent Bazemore" height="33px" onerror="this.onerror=null;this.src='/wp-content/uploads/NBA-Player-33x33.png';" src="/wp-content/uploads/kent-bazemore-33x33.png" title="Kent Bazemore" width="33px"/>Kent Bazemore</a></td>, <td><span class="roster-rating label label-gold label-default">76</span></td>, <td class="font-weight-normal">SG<!-- -->
</td>, <td class="hidden-xs font-weight-normal">6"5'</td>, <td class="font-weight-normal">Under Armour</td>]
Kent Bazemore SG
 6"5' Under Armour 76
[<td class="hidden-xs font-weight-normal">3</td>, <td class="roster-entry"><a href="https://nba2k19.2kratings.com/player/dewayne-dedmon"><img alt="Dewayne Dedmon" height="33px" onerror="this.onerror=null;this.src='/wp-content/uploads/NBA-Player-33x33.png';" src="/wp-content/uploads/dewayne-dedmon-33x33.png" title="Dewayne Dedmon" width="33px"/>Dewayne Dedmon</a>

,name,rating,team
0,Deyonta Davis,71,ATL


In [70]:
# modify the function to add the results to an existing df

def get_data(url, team):
  
  names, positions, heights, brands, injury, two_ways, all_stars, rookies, ratings = ([] for i in range(9))

  
  # request to the page
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')
  
  
  # get the data for each player in the table
  
  # table containing all players
  table = soup.find("tbody")


  for player in table.find_all("tr"):
    
    # name of the player
    name = player.find_all("td")[1].a.text
    names.append(name)


    # pos
    pos = player.find_all("td")[3].text
    positions.append(pos)

    # rating
    rating = player.find_all("td")[2].span.text
    ratings.append(rating)
    
    # height
    height =  player.find_all("td")[4].text
    heights.append(height)

    # brand
    brand = player.find_all("td")[5].text
    brands.append(brand)

    # check if all star 
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-star"})) > 0:
      all_star = 1
    else: 
      all_star = 0
    all_stars.append(all_star)

    # check if injured
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-plus-circle"})) > 0:
      injured = 1
    else: 
      injured = 0
    injury.append(injured)

    # check if two-way
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-arrows-h"})) > 0:
      two_way = 1
    else: 
      two_way = 0
    two_ways.append(two_way)

    # check if rookie
    if len(player.find_all("td")[1].find_all("small")) == 1:
      rookie = 1
    else:
      rookie = 0
    rookies.append(rookie)  
    
  teams =  [team for i in range(len(names))] 
  df_rat = pd.DataFrame({'name': names, 'team': teams, 'position': positions, 
                         'height' : heights, 'brand' : brands, 'all_star': all_stars, 
                         'injured' : injury, 'two_way' : two_ways, 'rookie': rookies,  
                         'rating': ratings})  
  return df_rat




In [71]:
get_data(url, 'ATL')

,name,team,position,height,brand,all_star,injured,two_way,rookie,rating
0,Kent Bazemore,ATL,SG\n,"6""5'",Under Armour,0,0,0,0,76
1,Dewayne Dedmon,ATL,C\n,"7""0'",Jordan,0,0,0,0,77
2,John Collins,ATL,PF\n,"6""10'",Adidas,0,0,0,0,84
3,Miles Plumlee,ATL,C\n,"6""11'",Peak,0,0,0,0,71
4,DeAndre’ Bembry,ATL,SF\n,"6""6'",Adidas,0,0,0,0,72
5,Taurean Prince,ATL,SF\n,"6""8'",Nike,0,0,0,0,77
6,Trae Young,ATL,PG\n,"6""2'",Adidas,0,0,0,1,84
7,Kevin Huerter,ATL,SG\n,"6""7'",Nike,0,0,0,1,74
8,Omari Spellman,ATL,PF\n,"6""9'",Nike,0,0,0,1,73
9,Justin Anderson,ATL,SF\n,"6""6'",Nike,0,0,0,0,72


In [74]:
def combine_df(url_teams):

    df_list = []
    for url in url_teams:
        df = get_data(url, dict_team_url[url])
        df_list.append(df)
    combined_df = pd.concat(df_list)
    return combined_df

In [75]:
combine_df(url_teams)

,name,team,position,height,brand,all_star,injured,two_way,rookie,rating
0,Kent Bazemore,Atlanta Hawks,SG\n,"6""5'",Under Armour,0,0,0,0,76
1,Dewayne Dedmon,Atlanta Hawks,C\n,"7""0'",Jordan,0,0,0,0,77
2,John Collins,Atlanta Hawks,PF\n,"6""10'",Adidas,0,0,0,0,84
3,Miles Plumlee,Atlanta Hawks,C\n,"6""11'",Peak,0,0,0,0,71
4,DeAndre’ Bembry,Atlanta Hawks,SF\n,"6""6'",Adidas,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...
11,Trevor Ariza,Washington Wizards,SF\n,"6""8'",Nike,0,0,0,0,76
12,Chasson Randle,Washington Wizards,PG\n,"6""2'",Nike,0,0,0,0,71
13,Bobby Portis,Washington Wizards,PF\n,"6""11'",Nike,0,0,0,0,79
14,Jabari Parker,Washington Wizards,PF\n,"6""8'",Jordan,0,0,0,0,79


In [53]:
url_teams

['https://nba2k19.2kratings.com/team/atlanta-hawks',
 'https://nba2k19.2kratings.com/team/boston-celtics',
 'https://nba2k19.2kratings.com/team/brooklyn-nets',
 'https://nba2k19.2kratings.com/team/charlotte-hornets',
 'https://nba2k19.2kratings.com/team/chicago-bulls',
 'https://nba2k19.2kratings.com/team/cleveland-cavaliers',
 'https://nba2k19.2kratings.com/team/dallas-mavericks',
 'https://nba2k19.2kratings.com/team/denver-nuggets',
 'https://nba2k19.2kratings.com/team/detroit-pistons',
 'https://nba2k19.2kratings.com/team/golden-state-warriors',
 'https://nba2k19.2kratings.com/team/houston-rockets',
 'https://nba2k19.2kratings.com/team/indiana-pacers',
 'https://nba2k19.2kratings.com/team/los-angeles-clippers',
 'https://nba2k19.2kratings.com/team/los-angeles-lakers',
 'https://nba2k19.2kratings.com/team/memphis-grizzlies',
 'https://nba2k19.2kratings.com/team/miami-heat',
 'https://nba2k19.2kratings.com/team/milwaukee-bucks',
 'https://nba2k19.2kratings.com/team/minnesota-timberwol

In [14]:
# modify the function to add the results to an existing df

def get_data(url):
  
  # request to the page
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')
  
  
  
  # get the data for each player in the table
  
  # table containing all players
  table = soup.find("tbody")





  for player in table.find_all("tr"):

    # name of the player
    name = player.find_all("td")[1].a.text
    names.append(name)

    # name of the team
    teams.append(dict_team_url[url])
    
#     team = player.find_all("td")[1].small.text
#     teams.append(team)

    # pos
    pos = player.find_all("td")[3].text
    positions.append(pos)

    # height
    height =  player.find_all("td")[4].text
    heights.append(height)

    # rating
    rating = player.find_all("td")[2].span.text
    ratings.append(rating)

    # brand
    brand = player.find_all("td")[5].text
    brands.append(brand)


    # check if all star 
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-star"})) > 0:
      all_star = 1
    else: 
      all_star = 0
    all_stars.append(all_star)

    # check if injured
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-plus-circle"})) > 0:
      injured = 1
    else: 
      injured = 0
    injury.append(injured)

    # check if two-way
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-arrows-h"})) > 0:
      two_way = 1
    else: 
      two_way = 0
    two_ways.append(two_way)


    # check if rookie
    if len(player.find_all("td")[1].find_all("small")) == 1:
      rookie = 1
    else:
      rookie = 0
    rookies.append(rookie)    


In [15]:
#start getting  list of the url where we can obtain all the data: one URL per team
url = 'https://nba2k19.2kratings.com/current-teams-on-nba-2k19'



# request to the page
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

teams = soup.find("ul", {"class": "list-group"})

url_teams = []
team_list = []

for team in teams.find_all('li'):
  url_teams.append(team.a['href'])
  team_list.append(team.a.text)
  


dict_team_url = {url_teams[i]:team_list[i] for i in range(0, len(team_list))}

print(dict_team_url)

{'https://nba2k19.2kratings.com/team/atlanta-hawks': 'Atlanta Hawks', 'https://nba2k19.2kratings.com/team/boston-celtics': 'Boston Celtics', 'https://nba2k19.2kratings.com/team/brooklyn-nets': 'Brooklyn Nets', 'https://nba2k19.2kratings.com/team/charlotte-hornets': 'Charlotte Hornets', 'https://nba2k19.2kratings.com/team/chicago-bulls': 'Chicago Bulls', 'https://nba2k19.2kratings.com/team/cleveland-cavaliers': 'Cleveland Cavaliers', 'https://nba2k19.2kratings.com/team/dallas-mavericks': 'Dallas Mavericks', 'https://nba2k19.2kratings.com/team/denver-nuggets': 'Denver Nuggets', 'https://nba2k19.2kratings.com/team/detroit-pistons': 'Detroit Pistons', 'https://nba2k19.2kratings.com/team/golden-state-warriors': 'Golden State Warriors', 'https://nba2k19.2kratings.com/team/houston-rockets': 'Houston Rockets', 'https://nba2k19.2kratings.com/team/indiana-pacers': 'Indiana Pacers', 'https://nba2k19.2kratings.com/team/los-angeles-clippers': 'Los Angeles Clippers', 'https://nba2k19.2kratings.com/t

In [0]:
names = []
teams = []
positions = []
heights = []
ratings = []
brands = []
injury = []
two_ways = []
all_stars = []
rookies = []

for url in url_teams:
  get_data(url)
  
# create dataframe
ratings_19 = pd.DataFrame({'name': names, 'team': teams, 'position': positions, 
              'height' : heights, 'brand' : brands,
              'all_star': all_stars, 'injured' : injury, 'two_way' : two_ways, 'rookie': rookies,  'rating': ratings})  

In [0]:
# Creamos el fichero
ratings_19.to_csv("2k19_ratings.csv", index=False)

# descargamos en formato .csv
files.download("2k19_ratings.csv")

# Stats data

In [80]:
# base URL of the site
url="https://www.basketball-reference.com"


# # robots.txt
# check_robots_txt(url) 

In [81]:
def get_stats(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')


  col_names = []
  for i in range(1,len(soup.tbody.find_all('tr')[1].find_all('td'))+1):
        col_names.append(soup.thead.find_all('th')[i].text.replace('%','.pct'))

  players = soup.tbody.find_all('tr')
  stats_df = pd.DataFrame(np.zeros((0,len(soup.tbody.find_all('tr')[1].find_all('td')))))

  # loop through the players
  for player in players:

    if player['class'] != ['thead']:

      # create list for every player
      player_as_list = []

      #loop through the player's stats
      for col in range(0,len(soup.tbody.find_all('tr')[1].find_all('td'))):
        player_as_list.append(player.find_all('td')[col].text)

      # obtain dataframe  
      df = pd.DataFrame(player_as_list)
      # append to the whole dataframe; transposed
      stats_df = stats_df.append(df.T)

  # modify column names and index
  stats_df.columns=col_names
  stats_df.index = np.arange(len(stats_df))
  
  # change blank spaces for NAs:
  stats_df = stats_df.replace(r'^\s*$', np.nan, regex=True)
  
  return stats_df

## Per Game stats

In [82]:
stats_1718 = get_stats('https://www.basketball-reference.com/leagues/NBA_2018_per_game.html')

In [83]:
stats_1718.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG.pct,...,FT.pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,24,OKC,75,8,15.1,1.5,3.9,.395,...,.848,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7
1,Quincy Acy,PF,27,BRK,70,8,19.4,1.9,5.2,.356,...,.817,0.6,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9
2,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,.629,...,.559,5.1,4.0,9.0,1.2,1.2,1.0,1.7,2.8,13.9
3,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,.512,...,.721,1.7,3.8,5.5,1.5,0.5,0.6,1.0,2.0,6.9
4,Arron Afflalo,SG,32,ORL,53,3,12.9,1.2,3.1,.401,...,.846,0.1,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4


## Advanced stats

In [0]:
stats_1718_adv = get_stats('https://www.basketball-reference.com/leagues/NBA_2018_advanced.html')

## Full stats: per game plus advanced

In [0]:
stats_1718_full = pd.merge(stats_1718, stats_1718_adv, how='inner', on=["Player", 'Pos', 'Age', 'Tm', 'G'])

stats_1718_full.columns = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MPG', 'FG', 'FGA', 'FG.pct',
       '3P', '3PA', '3P.pct', '2P', '2PA', '2P.pct', 'eFG.pct', 'FT', 'FTA',
       'FT.pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'MP_Total', 'PER', 'TS.pct', '3PAr', 'FTr', 'ORB.pct', 'DRB.pct', 'TRB.pct',
       'AST.pct', 'STL.pct', 'BLK.pct', 'TOV.pct', 'USG.pct', 'empty1', 'OWS',
       'DWS', 'WS', 'WS/48', 'empty2', 'OBPM', 'DBPM', 'BPM', 'VORP']
# drop columns with NA created with the scraping part
stats_1718_full  = stats_1718_full.drop(['empty1', 'empty2'], axis = 1)

In [0]:
# Creamos el fichero
stats_1718_full.to_csv("full_stats_1718.csv", index=False)

# descargamos en formato .csv
files.download("full_stats_1718.csv")